In [ ]:
# ==========================
# 必要ライブラリのインストール
# ==========================
!pip -q install scikit-image

In [ ]:
# ==========================
# ライブラリ読み込み
# ==========================
import numpy as np
import matplotlib.pyplot as plt
from skimage import data, img_as_float
from skimage.color import rgb2gray

# ノートブック用スタイル
plt.rcParams["figure.figsize"] = (12, 8)

In [ ]:
# ======================================
# 1. 画像読み込み
# ======================================
img_rgb = img_as_float(data.astronaut())    # RGB, 値域 0–1
img_gray = rgb2gray(img_rgb)               # グレースケールに変換
h, w = img_gray.shape

print(f"画像サイズ: {h}×{w}  （総画素 {h*w:,}）")
plt.imshow(img_gray, cmap="gray")
plt.title("Original (Grayscale)")
plt.axis("off")

In [ ]:
# ======================================
# 2. SVD → 低ランク近似を返す関数
# ======================================
def svd_low_rank(img: np.ndarray, k: int) -> np.ndarray:
    """
    img : 2D (float) 画像
    k   : 残すランク
    戻り値: rank-k 近似画像
    """
    U, s, Vt = np.linalg.svd(img, full_matrices=False)
    S_k = np.diag(s[:k])
    img_k = U[:, :k] @ S_k @ Vt[:k, :]
    return np.clip(img_k, 0, 1)            # 範囲を 0–1 に戻しておく

In [ ]:
# ======================================
# 3. 複数の k で可視化
# ======================================
ks = [5, 20, 50, 100, 200]     # 試したいランク
ncols = len(ks) + 1            # ＋オリジナル

fig, axes = plt.subplots(1, ncols, figsize=(3*ncols, 3))
axes[0].imshow(img_gray, cmap="gray")
axes[0].set_title("Original")
axes[0].axis("off")

for ax, k in zip(axes[1:], ks):
    img_k = svd_low_rank(img_gray, k)
    ax.imshow(img_k, cmap="gray")
    ax.set_title(f"rank = {k}")
    ax.axis("off")

fig.suptitle("Low-Rank Approximations (Grayscale)", fontsize=16)
plt.tight_layout()
plt.show()

In [ ]:
# ======================================
# 4. 画質 vs. 圧縮比の簡易評価
#    （PSNR = Peak Signal‑to‑Noise Ratio）
# ======================================
from skimage.metrics import peak_signal_noise_ratio as psnr

orig_size   = img_gray.size        # ピクセル数
print("k\t保存パラメータ数\t圧縮率\tPSNR[dB]")
for k in ks:
    params = k*(h + w + 1)         # U: hk, V: wk, s: k
    rate   = params / orig_size
    img_k  = svd_low_rank(img_gray, k)
    psnr_k = psnr(img_gray, img_k, data_range=1)
    print(f"{k:>3}\t{params:>12,}\t{rate:6.2%}\t{psnr_k:6.2f}")